# 一.项目背景与介绍

这是一份巴西电商平台的公共数据集，来源于Kaggle网站公开数据集，包含Olist Store电商平台多个市场2016年至2018年10万份真实订单的交易数据，数据集中提到的公司和合作伙伴已经被匿名化。本项目通过Python，MySQL数据库和Tableau来进行分析（Tableau用于可视化分析，Python和MySQL用于数据清洗和其他)，旨在探究Olist电商平台的交易情况，产品的分布，以及对用户圈定和细分，进行更精确的定向销售。


# 二.数据集准备&分析框架

## 数据集准备
* 分析语言：Python
* IDE：Jupyter Notebook
* 相关库：pandas、NumPy、Matplotlib、Seaborn
* 数据库：MySQL
* 可视化工具：Tableau
* 数据集：Brazilian E-Commerce Public Dataset by Olist
* 数据集大小：9张表，52列，10万多条数据
* 表名说明：
  * olist_orders_dataset：订单交易数据等信息
  * olist_order_items_dataset：每个订单中购买的商品信息
  * olist_order_reviews_dataset：客户所做评论的信息
  * olist_products_dataset：Olist销售的产品数据
  * product_category_name_translation：商品品类和商品品类的英文翻译
  * olist_customers_dataset: 顾客的id及位置等信息
  * olist_geolocation_dataset: 巴西邮政编码及其维度/经度坐标的信息
  * olist_order_payments_dataset: 订单付款的信息
  * olist_sellers_dataset: 完成订单的卖家信息

<img src="picture/database.png" width="70%">

* 字段说明（共38个字段）
  * 1.product_id：商品ID
  * 2.seller_id：商家ID
  * 3.order_id：订单ID
  * 4.customer_id：订单对应的用户ID
  * 5.order_status：订单状态
  * 6.order_purchase_timestamp：下单时间
  * 7.order_approved_at：付款审批时间
  * 8.order_delivered_carrier_date：订单过账日期
  * 9.order_delivered_customer_date：客户实际订单交货日期
  * 10.order_estimated_delivery_date：订单预计交货日期
  * 11.customer_unique_id：用户ID
  * 12.customer_zip_code_prefix：客户邮政编码前5位
  * 13.customer_city：客户所在城市
  * 14.customer_state：客户所在的州
  * 15.review_id：评论ID
  * 16.review_score：评价得分
  * 17.review_comment_title：评论标题（葡萄牙语）
  * 18.review_comment_message：评论内容（葡萄牙语）
  * 19.review_creation_date：发出满意度调查日期
  * 20.review_answer_timestamp：客户满意度回复日期
  * 21.payment_sequential：客户付款顺序
  * 22.payment_type：付款方式
  * 23.payment_installments：客户选择的分期付款数量
  * 24.payment_value：交易金额
  * 25.order_item_id：订单序号
  * 26.price：商品价格
  * 27.freight_value：运费
  * 28.seller_zip_code_prefix：卖家邮政编码前5位
  * 29.seller_city：卖家所在城市
  * 30.seller_state：卖家所在州
  * 31.product_category_name：类别名称
  * 32.product_name_lenght：产品名称长度
  * 33.product_description_lenght：产品说明长度
  * 34.product_photos_qty：产品照片数量
  * 35.product_weight_g：产品重量单位g
  * 36.product_length_cm：产品长度单位cm
  * 37.product_height_cm：产品高度单位cm
  * 38.product_width_cm：产品宽度单位cm

## 分析框架
根据以上数据集，可以从产品，销售，用户，商家四个维度来展开分析各项指标，分析框架如下：


<img src="picture/巴西Olist电商数据.png" width="90%">

# 三.数据预处理

## 导入数据&理解数据

In [276]:
# 导入分析包
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objs import Scatter
# 导入数据库
from sqlalchemy import create_engine  # 导入数据库
# 在jupyter notebook里面显示图表
% matplotlib inline
# 设置中文
from matplotlib.font_manager import _rebuild
_rebuild()
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

获取数据：9个子数据集通过MySQL建表合并后，导入合并数据集和9个子数据集

In [290]:
# 导入9个子数据集
orders = pd.read_csv('dataset/olist_orders_dataset.csv')
payments = pd.read_csv('dataset/olist_order_payments_dataset.csv')
customers = pd.read_csv('dataset/olist_customers_dataset.csv')
reviews = pd.read_csv('dataset/olist_order_reviews_dataset.csv')
order_items = pd.read_csv('dataset/olist_order_items_dataset.csv')
products = pd.read_csv('dataset/olist_products_dataset.csv')
seller = pd.read_csv('dataset/olist_sellers_dataset.csv')
geolocation = pd.read_csv('dataset/olist_geolocation_dataset.csv')
pro_trans = pd.read_csv('dataset/product_category_name_translation.csv')

## 清洗数据

### 用户维度的数据清洗和整理

In [5]:
# 选取用户维度数据子集：user_data
order_payment = pd.merge(orders, payments, on='order_id', how='left')
payorder_customer = pd.merge(order_payment, customers, on='customer_id', how='left')
user_data = pd.merge(payorder_customer, order_items, on='order_id', how='left')
user_data.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017/10/2 10:56,2017/10/2 11:07,2017/10/4 19:55,2017/10/10 21:25,2017/10/18 0:00,1.0,credit_card,...,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017/10/6 11:07,29.99,8.72
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017/10/2 10:56,2017/10/2 11:07,2017/10/4 19:55,2017/10/10 21:25,2017/10/18 0:00,3.0,voucher,...,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017/10/6 11:07,29.99,8.72
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017/10/2 10:56,2017/10/2 11:07,2017/10/4 19:55,2017/10/10 21:25,2017/10/18 0:00,2.0,voucher,...,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017/10/6 11:07,29.99,8.72


In [6]:
#用户数据是否有缺失值
missingTotal=user_data.isnull().sum()
missingExist=missingTotal[missingTotal>0]
missingExist=missingExist.sort_values(ascending=False)
print('缺失值计数统计如下：\n',missingExist)
print('-'*50)
missingExistPer=missingExist/user_data.shape[0]
missingExistPer=missingExistPer.apply(lambda x: format(x, '.2%'))
print('缺失值占比统计如下：\n',missingExistPer)

缺失值计数统计如下：
 order_delivered_customer_date    3397
order_delivered_carrier_date     2074
freight_value                     830
price                             830
shipping_limit_date               830
seller_id                         830
product_id                        830
order_item_id                     830
order_approved_at                 176
payment_value                       3
payment_installments                3
payment_type                        3
payment_sequential                  3
dtype: int64
--------------------------------------------------
缺失值占比统计如下：
 order_delivered_customer_date    2.87%
order_delivered_carrier_date     1.75%
freight_value                    0.70%
price                            0.70%
shipping_limit_date              0.70%
seller_id                        0.70%
product_id                       0.70%
order_item_id                    0.70%
order_approved_at                0.15%
payment_value                    0.00%
payment_installments        

In [7]:
# 删除缺失值
user_data.dropna(inplace=True)
# 删除重复值
user_data.drop_duplicates(inplace= True)
# 观察异常数据情况
user_data.describe()

,payment_sequential,payment_installments,payment_value,customer_zip_code_prefix,order_item_id,price,freight_value
count,115018.000000,115018.000000,115018.000000,115018.000000,115018.000000,115018.000000,115018.000000
mean,1.090699,2.936958,171.915654,35081.609113,1.196404,120.080129,19.995686
std,0.684409,2.772466,266.125730,29851.527722,0.699409,183.038922,15.744372
min,1.000000,0.000000,0.000000,1003.000000,1.000000,0.850000,0.000000
25%,1.000000,1.000000,60.850000,11310.000000,1.000000,39.900000,13.080000
50%,1.000000,2.000000,108.190000,24320.000000,1.000000,74.900000,16.280000
75%,1.000000,4.000000,188.940000,58801.750000,1.000000,133.000000,21.180000
max,26.000000,24.000000,13664.080000,99980.000000,21.000000,6735.000000,409.680000


* payment_installments分期最小值一般为1，表示不分期，或者为3、6、12等，上面显示0属于异常值。
* payment_value支付金额一般也不可能为0，但客户有可能使用了代金劵，这个后面看数据再确认是否异常。

In [8]:
user_data[user_data['payment_installments']==0]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
75796,744bade1fcf9ff3f31d860ace076d422,5e5794daaa13f73e2f1cdb4114529843,delivered,2018/4/22 11:34,2018/4/24 19:04,2018/4/24 3:14,2018/4/27 20:55,2018/5/16 0:00,2.0,credit_card,...,f54cea27c80dc09bfe07b1cf1e01b845,5263,sao paulo,SP,1.0,0cf573090c66bb30ac5e53c82bdb0403,7202e2ba20579a9bd1acb29e61fe71f6,2018/4/26 12:31,45.90,12.79
78915,1a57108394169c0b47d8f876acc9ba2d,48ebb06cf56dba9d009230cc751bb195,delivered,2018/5/15 16:25,2018/5/15 16:36,2018/5/17 12:37,2018/5/24 15:45,2018/6/6 0:00,2.0,credit_card,...,9925e1d7dff0d807355599dee04830ab,44001,feira de santana,BA,1.0,db35a562fb6ba63e19fa42a15349dc04,282f23a9769b2690c5dda22e316f9941,2018/5/18 16:31,41.69,23.28
78916,1a57108394169c0b47d8f876acc9ba2d,48ebb06cf56dba9d009230cc751bb195,delivered,2018/5/15 16:25,2018/5/15 16:36,2018/5/17 12:37,2018/5/24 15:45,2018/6/6 0:00,2.0,credit_card,...,9925e1d7dff0d807355599dee04830ab,44001,feira de santana,BA,2.0,db35a562fb6ba63e19fa42a15349dc04,282f23a9769b2690c5dda22e316f9941,2018/5/18 16:31,41.69,23.28


可以看到，上述3条异常数据的payment_value支付金额不为0，而且都使用credit_card 借记卡支付，所以payment_installments分期为0的3条数据为异常值，可以删掉。

In [9]:
user_data[user_data['payment_value']==0]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
503,45ed6e85398a87c253db47c2d9f48216,8eab8f9b3c744b76b65f7a2c0c8f2d6c,delivered,2017/6/8 21:18,2017/6/8 21:30,2017/6/9 8:35,2017/6/22 17:44,2017/7/10 0:00,3.0,voucher,...,e4000306cf2f63714e6bb70dd20a6592,74595,goiania,GO,1.0,75d6b6963340c6063f7f4cfcccfe6a30,cc419e0650a3c5ba77189a1882b7556a,2017/6/15 21:30,56.99,14.15
38539,6ccb433e00daae1283ccc956189c82ae,843b211abe7b0264dd4a69eafc5bdf43,delivered,2017/10/26 23:51,2017/10/27 0:46,2017/10/27 19:53,2017/11/8 18:47,2017/11/22 0:00,4.0,voucher,...,677ad9cdca6c47c733f0cc6e23d7bb4c,25625,petropolis,RJ,1.0,2ee4be9805f228255a38a6f9b03fff1a,da20530872245d6cd9d2f5725613c430,2017/11/3 0:46,94.00,28.04
102033,b23878b3e8eb4d25a158f57d96331b18,648121b599d98c420ef93f6135f8c80c,delivered,2017/5/27 19:20,2017/5/27 19:35,2017/5/29 4:49,2017/6/7 8:53,2017/6/29 0:00,4.0,voucher,...,9077449283dc1319d5e51fb3159b28e2,29230,anchieta,ES,1.0,e306ca54c91b21392317d5b4632c9fe3,e49c26c3edfa46d227d5121a6b6e4d37,2017/6/1 19:35,135.30,36.27
115449,8bcbe01d44d147f901cd3192671144db,f2def7f64f36952f2f5a9791f0285f34,delivered,2018/1/24 23:24,2018/1/24 23:37,2018/1/30 22:43,2018/2/2 21:26,2018/2/21 0:00,4.0,voucher,...,cc5aa1b8337e394da54980226bb0f6d4,21040,rio de janeiro,RJ,1.0,85d4c1a46f08f730de651ea6f6645313,3d871de0142ce09b7081e2b9d1733cb1,2018/1/30 23:37,59.00,15.16


上面4条数据payment_value支付金额为0，是因为使用了voucher代金劵支付，不属于异常值，不用处理。

In [10]:
# 删除3条异常值
user_data = user_data.drop(index=user_data[user_data['payment_installments']==0].index)
# 数据类型转化：把字符串转化为日期时间数据类型
def transform_datetime(data, colum_list):
    for i in colum_list:
        data[i] = pd.to_datetime(data[i])
    print('数据类型转化成功！')
colum_list = ['order_purchase_timestamp', 'order_approved_at',
              'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
transform_datetime(user_data,colum_list) 

数据类型转化成功！


In [11]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115015 entries, 0 to 118433
Data columns (total 22 columns):
order_id                         115015 non-null object
customer_id                      115015 non-null object
order_status                     115015 non-null object
order_purchase_timestamp         115015 non-null datetime64[ns]
order_approved_at                115015 non-null datetime64[ns]
order_delivered_carrier_date     115015 non-null datetime64[ns]
order_delivered_customer_date    115015 non-null datetime64[ns]
order_estimated_delivery_date    115015 non-null datetime64[ns]
payment_sequential               115015 non-null float64
payment_type                     115015 non-null object
payment_installments             115015 non-null float64
payment_value                    115015 non-null float64
customer_unique_id               115015 non-null object
customer_zip_code_prefix         115015 non-null int64
customer_city                    115015 non-null object
custome

经过缺失值处理、异常值处理和数据类型转化等数据清洗操作，user_data数据集大小为：22列，11505行。接下来把user_data数据集导出到MySQL数据库，用Tableau连接MySQL数据库进行可视化数据分析。

In [22]:
# 导出清洗后的数据到MySQL数据库
from sqlalchemy import create_engine
def export_mysql(data, user, password, host_port, db, table_name):
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(user, password, host_port, db))
    con = engine.connect() # 创建连接
    data.to_sql(table_name, engine, if_exists='replace', index=False)
    print('导出成功!')
user = 'root'
password = 'Lori1108'
host_port = 'localhost:3306'
db = 'Olist2018'
data = user_data
table_name = 'user_data'

export_mysql(data, user, password, host_port, db, table_name)
# con.close()

导出成功!


### 商家维度的数据清洗和整理

In [153]:
# 合并数据集并从中选取所需字段
seller_df1 = pd.merge(user_data,reviews, on='order_id',how='left')
seller_df2 = pd.merge(seller_df1,seller, on='seller_id',how='left')
seller_data = seller_df2[['order_id','seller_id','customer_unique_id','order_purchase_timestamp',
                           'payment_value','order_delivered_carrier_date','order_estimated_delivery_date',
                           'review_id','review_score','review_creation_date','review_answer_timestamp',
                          'seller_zip_code_prefix','seller_city','seller_state']]
# 删除缺失值
seller_data.dropna(inplace=True)
# 删除重复值
seller_data.drop_duplicates(inplace= True)

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [154]:
#计算商家送货时间
d1=seller_data['order_estimated_delivery_date']
d2=seller_data['order_delivered_carrier_date']
seller_data['delivery_days']=(d1 - d2).apply(lambda x : x.days)

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [155]:
# 数据类型转化
column_list =['order_purchase_timestamp','order_delivered_carrier_date','order_estimated_delivery_date',
              'review_creation_date','review_answer_timestamp'] 
transform_datetime(seller_data,column_list)

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



数据类型转化成功！


In [156]:
seller_data.head()

,order_id,seller_id,customer_unique_id,order_purchase_timestamp,payment_value,order_delivered_carrier_date,order_estimated_delivery_date,review_id,review_score,review_creation_date,review_answer_timestamp,seller_zip_code_prefix,seller_city,seller_state,delivery_days
0,e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:00,18.12,2017-10-04 19:55:00,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,2017-10-11,2017-10-12 03:43:00,9350,maua,SP,13
1,e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:00,2.00,2017-10-04 19:55:00,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,2017-10-11,2017-10-12 03:43:00,9350,maua,SP,13
2,e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 10:56:00,18.59,2017-10-04 19:55:00,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,2017-10-11,2017-10-12 03:43:00,9350,maua,SP,13
3,53cdb2fc8bc7dce0b6741e2150273451,289cdb325fb7e7f891c38608bf9e0962,af07308b275d755c9edb36a90c618231,2018-07-24 20:41:00,141.46,2018-07-26 14:31:00,2018-08-13,8d5266042046a06655c8db133d120ba5,4,2018-08-08,2018-08-08 18:37:00,31570,belo horizonte,SP,17
4,47770eb9100c2d0c44946d9cf07ec65d,4869f7a5dfa277a7dca6462dcf3b52b2,3a653a41f6f9fc3d2a113cf8398680e8,2018-08-08 08:38:00,179.12,2018-08-08 13:50:00,2018-09-04,e73b67b67587f7644d5bd1a52deb1b01,5,2018-08-18,2018-08-22 19:07:00,14840,guariba,SP,26


In [157]:
seller_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102160 entries, 0 to 115707
Data columns (total 15 columns):
order_id                         102160 non-null object
seller_id                        102160 non-null object
customer_unique_id               102160 non-null object
order_purchase_timestamp         102160 non-null datetime64[ns]
payment_value                    102160 non-null float64
order_delivered_carrier_date     102160 non-null datetime64[ns]
order_estimated_delivery_date    102160 non-null datetime64[ns]
review_id                        102160 non-null object
review_score                     102160 non-null int64
review_creation_date             102160 non-null datetime64[ns]
review_answer_timestamp          102160 non-null datetime64[ns]
seller_zip_code_prefix           102160 non-null int64
seller_city                      102160 non-null object
seller_state                     102160 non-null object
delivery_days                    102160 non-null int64
dtypes: dat

经过缺失值处理、异常值处理和数据类型转化等数据清洗操作，seller_data数据集大小为：14列，102160行。

In [124]:
#导出到mysql数据库
data = seller_data
table_name = 'seller_data'
export_mysql(data,user,password,host_port,db,table_name)

导出成功!


### 产品维度的数据清洗和整理

In [146]:
# 合并相关数据集并选相关字段
ba_pro = pd.merge(seller_df2,products, on='product_id',how='left')
product_df = pd.merge(ba_pro,pro_trans, on='product_category_name',how='left')
product_data = product_df[['order_id','order_item_id','customer_unique_id','product_id',
                         'product_category_name_english','price','freight_value','payment_value','order_status','order_purchase_timestamp']]
# 删除缺失值
product_data.dropna(inplace=True)
# 删除重复值
product_data.drop_duplicates(inplace= True)

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [147]:
# 数据类型转化
column_list =['order_purchase_timestamp'] 
transform_datetime(product_data,column_list)

数据类型转化成功！


/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [148]:
product_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112722 entries, 0 to 115707
Data columns (total 10 columns):
order_id                         112722 non-null object
order_item_id                    112722 non-null float64
customer_unique_id               112722 non-null object
product_id                       112722 non-null object
product_category_name_english    112722 non-null object
price                            112722 non-null float64
freight_value                    112722 non-null float64
payment_value                    112722 non-null float64
order_status                     112722 non-null object
order_purchase_timestamp         112722 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 9.5+ MB


经过缺失值处理、异常值处理和数据类型转化等数据清洗操作，product_data数据集大小为：10列，112722行。

In [149]:
#导出到mysql数据库
data = product_data
table_name = 'product_data'
export_mysql(data,user,password,host_port,db,table_name)

导出成功!


### 销售维度的数据清洗和整理

In [256]:
# 合并筛选销售维度的数据集
sale_data=seller_data[['order_id','payment_value','customer_unique_id',
                     'order_purchase_timestamp','order_delivered_carrier_date','order_estimated_delivery_date',
                    'seller_city','seller_state','delivery_days']]
#计算客单价
sale_data['average_price']=15831410/102160

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [257]:
#导出到mysql数据库
data = sale_data
table_name = 'sale_data'
export_mysql(data,user,password,host_port,db,table_name)

导出成功!


# 四.数据可视化分析

## 用户维度
### 1）从「用户」维度分析「用户」的基础属性，行为偏好和交易属性。
* 基础属性：
  * 用户地理位置分布情况
  * 城市购买力分布
* 行为偏好：
  * 各时段订单量和销售额分布
  * 各工作日订单量和销售额分布
* 交易属性：
  * 支付偏好（占比）
  * 分期偏好（占比）

<img src="Chart/基础属性.png" width="90%">

从上图可知：

* 用户地理位置分布情况：用户主要集中在7个州和11个城市（州用户数量>3000，市客户数量>1000），巴西圣保罗（SP）、里约热内卢（RJ）、米纳斯吉拉斯州(MG)这三个州的客户数位于前列。其中，巴西圣保罗(SP)是用户数量最多的州，占用户的42.18%，而用户数分布最多的城市是巴西圣保罗(SP)的sao paulo(圣保罗)。

* 城市购买力排名：购买力排名前三强为sao paulo(圣保罗)、rio de janeiro(约热内卢)、Belo Horizonte（贝洛奥里藏特)

<img src="Chart/行为偏好.png" width="80%">

* 各时段订单量和销售额分布： 用户下单时间主要集中在早上9点到晚上22点，且下午14点~16点购买力最强。
* 各工作日订单量和销售额分布： 工作日的订单量和销售额表现优于周末。

* 用户购物时间偏好： 用户喜欢在晚上和工作日购物。

<img src="Chart/交易属性.png" width="90%">

* 用户付款方式偏好： 115015个订单中，用户付款方式首选信用卡credit_card支付，占比高达73.81%，其次是通过线下付款方式boleto(巴西主流的线下支付工具)，占比高达19.43，代金劵vouche和借记卡debit_card分别占比5.32%和1.44%。
* 用户分期偏好：近半数用户选择分期付款的方式，大多集中在2~3期，一定程度上说明该平台是比较推荐分期付款的方式。

### 2) 基于用户的属性和行为偏好，进一步分析用户价值

* 用户价值（用户人数分布和交易金额占比）：
利用RFM模型对用户价值进行量化，依据用户的最近一次消费时间，消费频率，消费金额来给用户打分，并将用户分为以下几类：

<img style="float: left;" src="picture/RFM.png" width="65%">

In [38]:
# 创建RFM模型进行分类处理
# 用户最近一次购买
r=user_data.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
r['R']=(pd.to_datetime('2018-10-18')-r['order_purchase_timestamp']).dt.days
r=r[['customer_unique_id','R']]
user_data['date']=user_data['order_purchase_timestamp'].astype(str).str[:10]
# 用户的消费频率
dup=user_data.groupby(['customer_unique_id','date'])['order_purchase_timestamp'].count().reset_index()
f=dup.groupby('customer_unique_id')['order_purchase_timestamp'].count().reset_index()
f.columns=['customer_unique_id','F']
# 用户的消费金额
summ=user_data.groupby('customer_unique_id')['payment_value'].sum().reset_index()
summ.columns=['customer_unique_id','payment_value']
comm=pd.merge(summ,f,left_on='customer_unique_id',right_on='customer_unique_id',how='inner')
comm['M']=comm['payment_value']/comm['F']
# 合并筛选出的子集
rfm=pd.merge(r,comm,on='customer_unique_id',how='inner')
# 清洗数据并计算
rfm['R_SCORE']=pd.cut(rfm['R'],5,labels=[5,4,3,2,1],right=False).astype(float)
rfm['F_SCORE']=pd.cut(rfm['F'],bins=[1,2,3,4,5,100000],labels=[1,2,3,4,5],right=False).astype(float)
rfm['M_SCORE']=pd.cut(rfm['M'],5,labels=[1,2,3,4,5],right=False).astype(float)
rfm['r_aboveavg']=(rfm['R_SCORE']>rfm['R_SCORE'].mean())*1
rfm['f_aboveavg']=(rfm['F_SCORE']>rfm['F_SCORE'].mean())*1
rfm['m_aboveavg']=(rfm['M_SCORE']>rfm['M_SCORE'].mean())*1
rfm['SCORE']=(rfm['r_aboveavg']*100)+(rfm['f_aboveavg']*10)+(rfm['m_aboveavg']*1)
# 构建RFM模型
def transform_l(x):
    if x==111:
        label='important_value_customer'
    elif x==110:
        label='consume_potential_customers'
    elif x==101:
        label='frequently_cultivate_customers'
    elif x==100:
        label='new_customers'
    elif x==11:
        label='important_value_loss_early_warning_customers'
    elif x==10:
        label='general_customers'
    elif x==1:
        label='high_consumption_comebake_customers'
    elif x==0:
        label='lost_customers'
    return label
rfm['type']=rfm['SCORE'].apply(transform_l)
rfm['type'].value_counts()

new_customers                          57587
lost_customers                         33731
consume_potential_customers             1510
general_customers                        504
high_consumption_comebake_customers        4
frequently_cultivate_customers             3
Name: type, dtype: int64

经过RFM模型的分类处理之后，将rfm数据集导出到MySQL数据库，用Tableau统计每一类用户的数量和创造的价值来进行分析。

In [35]:
#导出到mysql数据库
data = rfm
table_name = 'RFM_data'
export_mysql(data,user,password,host_port,db,table_name)

导出成功!


<img style="float: left;" src="Chart/RFM_table.png" width="35%">

<img src="Chart/RFM.png" width="90%">

   * Olist电商平台大部分用户都是新用户和流失用户，重要价值用户和频率深耕用户数量极少。
   * 用户的流失率较高，缺少一般维持用户这一现状对平台的维持和运营及其不利，但是新用户数量较多说明了近期的拉新工作很有进展，接下来用怎样的手段让新用户留存，提高留存率的工作至关重要。
   * 用户分层的方式，不仅可以统计出平台的运营所处的阶段水平，还可以对每一层的用户进行更详细的用户画像研究，从而针对不同类型的用户的行为和价值对产品做出提升和改进。


## 商家维度

### 1）从「商家」维度，可以分析「商家」的地区分布，交易量，销售额，商家送货时间和评分指标
* 地区分布
* 交易量
* 销售额
* 商家送货时间（从买家下单一直到收到货的时间）
* 评分指标：
  * 平均评分（满意度）
  * 用户评分占比
  * 各时段买家评价数分布



<img src="Chart/商家3.png" width="35%">

<img style="float: left;" src="Chart/商家1.png" width="90%">

* 商家的交易量，销售额分布：自2017年开始，年交易量剧增，商家大多数来自于SP，商家的地区分布比例与用户的地区分布不完全相同，主要体现在来自SP的商家比例远多于用户比例，说明这个地区可能不仅人口密度较高，商业贸易也比较发达。
* 商家送货时间：送货时间大致呈正态分布，大多数的送货时间集中在15-25天的范围内，均值为20左右，该平台的用户基本上可以在20天左右收到商品。

<img src="Chart/商家2.png" width="90%">

* 用户填写满意度的高峰期一般：在早上10点~下午1点，晚上22点~第二天凌晨1点。
* 用户对该电商平台满意度较高，好评率高达70.87%。用户满意度较高；但1分和2分合计占比12.4%左右，平台还有一定的提高空间。 

### 2）评分相关特征
利用随机森林分析商家的地区，交易量，交易额，送货时间这三个特征与商家评分的相关性。

In [208]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
seller_data['order_id']=LabelEncoder().fit_transform(seller_data['order_id'])
#取得特征
col = ['order_id','seller_state','payment_value','delivery_days']
X = seller_data[col].values
#取得标签
Y = seller_data['review_score'].values
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size=0.3) 
rfc = RandomForestClassifier() 
rfc = rfc.fit(Xtrain,Ytrain.astype('int')) 
score_r = rfc.score(Xtest,Ytest) 
score_r

/Users/weiminpang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



0.49660663012268336

In [206]:
rfc.feature_importances_

array([0.41319287, 0.03472019, 0.39564323, 0.15644371])

模型的拟合程度一般，达到了0.49.
四个feature分别为商家的地区，交易量，交易额，送货时间。结果说明就这四个特征而言，与评分关系最大的特征是交易量，其次是商家的交易额。这能说明这些特征与评分结果的相关性程度。
但由于模型的拟合程度一般，这些特征并不能作为充分必要条件。

## 产品维度

### 从「产品」维度，可以分析「产品」的订单量Top10产品，销售额Top10产品和各类商品贡献分布
* 订单量前十的产品
* 销售额前十的产品
* 各类商品贡献度分布

<img style="float: left;"  src="Chart/产品一.png" width="25%">

该平台商品共有52类，从产品的种类订单量上初步来看，订单量排名前十的产品中大多数是家具家用类，医美类，运动类，电子类等。
排名第一的是床上用品、洗漱用品、桌子类的产品，订单量为11552个。

<img src="Chart/产品二.png" width="90%">

销售额TOP10的产品种类占总销售额的64.57%，TOP20的产品种类占总销售额的88.43%。
头部效应明显，销量额呈严重的两极分化；极大部分的子类产品销量基本趋近于0，TOP10类产品销量遥遥领先。

## 销售维度

### 从「销售」维度，可以分析「销售」的地区维度，整体维度，时间维度和订单交付维度的销售情况。
* 地区维度：各地区销售量
* 整体维度：总销售额，总销售量，客单价（总销售额/交易总人数）
* 时间维度：年，月，季交易量/销售额
* 订单交付维度：
  * 订单平均交付时长（总收货时长/总订单数）
  * 订单延期交付率（延期的订单数/总订单数）
  * 延期交付订单的平均延期时长（总延期时长/延期的订单数）
  
1）地区维度

<img src="Chart/销售-地区.png" width="80%">

Olist平台以自东向西的地理方向来展开业务，其销售的最佳位置主要在SP这个州，其次是于其相邻的MG与PR。
总体上来看来自SP的销量远高于其他所有州，而其他州的销量则不相上下。
这与巴西的人口密度有很大关系，同时也存在着一定的地区差异性。（概图从浅绿色到深蓝色，颜色越深，销量越高）

2）整体维度&时间维度

<img src="Chart/销售1.png" width="90%">

* 从销售和交易量上看，Olist平台的交易业务是逐渐缓步上升的状态（2018年第三季度因为九~十二月没有数据显得较低，若加上预测数据，应比上一季度稍微高出一些），说明经营状态是比较良好的和稳定的，有上升的趋势。
* 另外，可以看出2017年的11月份是一个交易的拐点，不仅是数值的最高点，在此之后，平台的交易量也提高了一个层次水平，并且稳步发展。
* 从季度维度观察到2018年第二季度（如图异常A）销售额增加，但交易量反倒减少。下面细分月份分析异常原因：

<img src="Chart/销售2.png" width="90%">

* 把月交易量的时间拆分，计算每周的交易额，发现2018年四月和八月销售额增加，交易量反倒减少的原因，主要异常集中在八月的第34周（异常D），四月（异常B）在周数维度无明显特征。
* 2017年11月份的交易额主要在第三周达到了最高，而其他前面的数字跟之前没有太大的差异。
* 下面详细分析2017年11月（19~25号）销售额和交易量上升的原因，2018年8月（19~25号）销售额和交易量异常的原因：

<img src="Chart/销售3.png" width="90%">

* 将时间段再次拆分，便不难发现，2017年11月24日这一天交易数字异常的高，是平时的3-4倍，之后的数字比之前略微偏高，但也相对正常。经过查询日期可以知道11月24日是著名的“黑五”，在这一天几乎全欧美国家的商品都会进行打折促销活动。而Olist平台也就是在这一天制定了合理的营销策略，吸引众多用户前来购买，从而带来了较大的经济效益，进而给该平台带来了更多的新用户，使之后的交易额可以较稳定的上升。
* 2018年8月25号的销售额和交易量异常相差最大，8月底到九月初是每年苹果电子产品发售新款的时间段，可能由于电子等产品的热度导致。



4）订单交付维度
* 订单平均交付时长近13天，巴西网购的平均交付时间过长
* 订单平均交付时长=总收货时长/总订单数

<img style="float: left;" src="Chart/销售4.png" width="20%">

* 订单延期交付率5.64%
* 订单延期交付率=延期的订单数/总订单数

<img style="float: left;" src="Chart/销售5.png" width="20%">

* 延期交付订单的平均延期时长为0.66天
* 延期订单的平均延期时长=总延期时长/延期的订单数

<img style="float: left;" src="Chart/销售6.png" width="20%">

# 五.总结与建议

* 用户维度：
  * 用户留存率：Olist电商平台处于成长期阶段,主要的目标应该是获取更多新用户，并将功能进一步优化。但是新用户数量虽多，但是用户的流失率也很高，留存率低，缺少忠实用户是该平台的一大缺点。建议在拉新的同时，优化功能，以提高用户的留存率。
  * 用户转化率：巴西用户更倾向于在工作日、工作时间内去Olist购物，平均的下单量少，仅有1.04。相当于很多用户复购率低。这与用户的高流失率有很大的关系，根本的原因应该是出在功能的使用。建议根据购买下单的流程进行追踪，算出每一个环节的转化率，对于转化率低的过程分析内在原因，并将此功能进行合理的改进。增加用户使用的频率，提高用户活跃度，才能使用户群体稳定留存，创造出更大的价值。
  * 支付方式：近七成的用户选择2~3期信用卡分期支付，建议合理设定免息门槛和分期利息，与用户使用较多的付款方式对应的第三方支付机构洽谈或自己申请开展支付业务，提高支付速度，简化用户网购流程，不仅可以促进用户消费还可以带来一定的收入。


* 商家维度：
  * 商家运营分布：Olist平台在地域上有一定的局限性，无论是商家还是用户分布在北方的明显很少，而大多数都聚集在SP和周围的几个州，地区两级分化现象严重，这与该平台的运营规模和运营方式有很大的关系。建议为了提升经济价值，扩大运营的规模，根据不同地区的消费特点，增加SP以外的其他地区的业务投入，加大宣传折扣力度，引入更多的商家和用户。
  * 用户评价：用户对网站整体好评率高达70.87%，对产品较为认可；商家的销售额和交易量影响用户评分，但平均12天左右的送货时间不影响整体评分，可能与巴西整个电商市场的运输常态有关。建议在用户第一次购物时给予积分、优惠券等奖励，同时推出会员卡储值消费，提高用户粘性。


* 产品维度：
  * 产品优化：Olist平台商品共有52类，符合二八分布，如销售额TOP10的产品种类总销售额的64.57%， 头部效应明显，销量额呈严重的两极分化。建议对需求旺盛的产品保持并进一步丰富，同时对畅销的头部产品进行分析，发掘用户需求特性，对平台的产品进行结构性优化。

* 销售维度：
  * 完善销售策略：2017年Olist平台于上升期，销售额、订单量呈上升状态，2018、2019年趋于平稳，在“黑五”的时候达到峰值。建议持续发挥该平台“黑五”的优势，完善优化销售和宣传策略，为平台带来更多的用户和经济效益。




对于该平台下一步主要的业务发展方向，建议优化相应的功能，增加用户的活跃性。
